Real Weather Agent 

In [ ]:
WEATHER_CODE_MAP = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    75: "Heavy snow",
    95: "Thunderstorm",
    99: "Thunderstorm with hail"
}


In [ ]:
desc = WEATHER_CODE_MAP.get(code, f"Unknown weather code {code}")
# fallback

1 - First try: Send the request.

2 - If it fails: Wait a short time, then try again.

3 - After repeated failures: Stop retrying and raise an error.

In [ ]:
import time, requests
from requests.exceptions import RequestException, Timeout

In [ ]:
HTTP_TIMEOUT_SECS = 8
RETRY_ATTEMPTS = 2
RETRY_BACKOFF_SECS = 0.6

def _request_with_retries(method: str, url: str, **kwargs) -> requests.Response:
    """Make an HTTP request with retries and backoff."""
    last_exc = None
    for attempt in range(RETRY_ATTEMPTS + 1):
        try:
            return requests.request(method, url, timeout=HTTP_TIMEOUT_SECS, **kwargs)
        except (RequestException, Timeout) as exc:
            last_exc = exc
            if attempt < RETRY_ATTEMPTS:
                time.sleep(RETRY_BACKOFF_SECS * (attempt + 1))
            else:
                raise last_exc

Define tools for geocoding and forecasts

In [ ]:
OPEN_METEO_GEOCODE_URL = "https://geocoding-api.open-meteo.com/v1/search"
OPEN_METEO_FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

In [ ]:
def geocode_city(name: str) -> dict:
    """Look up latitude/longitude for a city using Open-Meteo."""
    params = {"name": name, "count": 1, "format": "json"}
    resp = _request_with_retries("GET", OPEN_METEO_GEOCODE_URL, params=params)
    data = resp.json()
    results = data.get("results") or []
    if not results:
        raise ValueError(f"Could not geocode city '{name}'.")
    r0 = results[0]
    return {"city": r0["name"], "lat": r0["latitude"], "lon": r0["longitude"]}

In [ ]:
def current_weather(lat: float, lon: float) -> dict:
    """Fetch current weather for coordinates using Open-Meteo."""
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": ["temperature_2m", "weather_code", "wind_speed_10m"],
        "timezone": "auto",
    }
    resp = _request_with_retries("GET", OPEN_METEO_FORECAST_URL, params=params)
    data = resp.json()
    cur = data.get("current")
    if not cur:
        raise ValueError("No weather data returned.")
    return {
        "temperature": cur["temperature_2m"],
        "weather_code": cur["weather_code"],
        "windspeed": cur["wind_speed_10m"],
    }


raw o/p = {"temperature": 16.4, "weather_code": 63, "windspeed": 3.1}

formatter that converts structured data into one-line human summaries

Paris: Moderate rain, 16°C, wind 3.1 m/s

In [ ]:
def format_weather_summary(city: str, payload: dict) -> str:
    code = payload["weather_code"]
    desc = WEATHER_CODE_MAP.get(code, f"Unknown weather code {code}")
    temp_c = payload["temperature"]
    wind = payload["windspeed"]
    return f"{city}: {desc}, {round(temp_c)}°C, wind {round(wind, 1)} m/s"

LangGraph WorkFLow

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage

In [ ]:
class MyMessagesState(MessagesState):
    pass

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
# Bind the real API tools we defined earlier
llm_with_tools = llm.bind_tools([geocode_city, current_weather])

from langchain_core.messages import AIMessage

def tool_calling_llm(state: MyMessagesState):
    system = SystemMessage(content=(
        "You are a helpful weather assistant. "
        "When the user mentions cities, call geocode_city for each city, then call current_weather. "
        "Prefer Celsius unless the user explicitly requests Fahrenheit/imperial."
    ))
    response = llm_with_tools.invoke([system] + state["messages"])
    return {"messages": [response]}

In [ ]:
def compose_final_answer(state: MyMessagesState):
    system = SystemMessage(content=(
        "Summarize any fetched weather results in plain language. "
        "Output one line per city, with condition, temperature, and wind. "
        "If any city failed, acknowledge it clearly instead of guessing."
    ))
    response = llm.invoke([system] + state["messages"])
    return {"messages": [response]}

START → LLM → tools → compose → END

In [ ]:
builder = StateGraph(MyMessagesState)

# Nodes
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([geocode_city, current_weather]))
builder.add_node("compose_final", compose_final_answer)

# Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
    {
        "tools": "tools",
        END: "compose_final",   # when no tool calls remain, go to final composer
    },
)
builder.add_edge("tools", "tool_calling_llm")  # loop back after tool execution
builder.add_edge("compose_final", END)

graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
def run_case(prompt: str, title: str):
    print("\n" + "=" * 65)
    print(title)
    print("=" * 65)
    res = graph.invoke({"messages": [HumanMessage(content=prompt)]})
    for m in res["messages"]:
        try:
            m.pretty_print()
        except Exception:
            role = getattr(m, "type", "message").upper()
            print(f"[{role}] {getattr(m, 'content', m)}")

In [ ]:
run_case("Weather in Paris and London please.", "Test 1: Two-city request (Paris + London)")
run_case("What's the weather in Tokyo?", "Test 2: Single city request (Tokyo)")
run_case("Show me the weather in New York in Fahrenheit.", "Test 3: Units preference (Fahrenheit)")
run_case("Weather for Delhi, Mumbai, and Kolkata.", "Test 4: Multi-city (India)")